In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
from keras.datasets import mnist
import sys
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, SpatialDropout2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K

from google.colab import drive
drive.mount("/content/gdrive")

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean and variance of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns
        Mean and variance of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0]))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:] = predictions.flatten()
    prediction = result.mean(axis=0)
    var = result.var(axis=0)
    return [prediction,var]

def run_model (X_train, y_train, X_pool_cand):
    """Initializes and trains a model from scratch on the given training data, returning test MAE and MSE.
    #Arguments
        X_train: training model inputs
        y_train: training model outputs
        X_pool_cand: model inputs in the pool that are included in X_{sample} and are candidates for acquisition
    #Returns
        Test MAE, MSE, and MC dropout sample variance for predictions on X_pool_cand
    """
    nb_pool = 3
    dropout_prob = 0.5
    nb_filters = 50
    nb_conv_init = 4 
    nb_conv_agg = 3
    nb_strides_init = 3
    nb_strides_agg = 2
    img_rows = 200
    img_cols = 200
    Weight_Decay = 2.0/len(y_train)

    #Keras Model
    model = Sequential()
    model.add(Convolution2D(nb_filters, nb_conv_init,  strides=nb_strides_agg, data_format="channels_last", input_shape=(img_rows, img_cols,3)))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, nb_conv_agg, strides=nb_strides_agg, data_format="channels_last"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last"))

    model.add(Convolution2D(nb_filters*2, nb_conv_agg, strides=1, data_format="channels_last"))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters*2, nb_conv_agg, strides=1, data_format="channels_last"))
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool), data_format="channels_last"))
    model.add(Dropout(dropout_prob))

    model.add(Flatten())
    model.add(Dense(150, W_regularizer=l2(Weight_Decay)))
    model.add(Activation('relu'))
    model.add(Dropout(dropout_prob))
    model.add(Dense(1, W_regularizer=l2(Weight_Decay)))
    model.compile(loss='mean_squared_error', optimizer='adam')
    hist = model.fit(X_train, y_train, batch_size=128, epochs=250, verbose=0)
    f = K.function([model.layers[0].input, K.learning_phase()], 
               [model.layers[-1].output])
    #Evaluating model on test data
    pool_prediction_results_1 = predict_with_uncertainty(f, X_pool_cand[0:2500], n_iter=50)
    pool_prediction_results_2 = predict_with_uncertainty(f, X_pool_cand[2500:], n_iter=50)
    pool_cand_variance = np.concatenate((pool_prediction_results_1[1], pool_prediction_results_2[1]))
    prediction_results = predict_with_uncertainty(f, X_test, n_iter=300)
    predicted_mean = prediction_results[0]
    return [np.mean(np.abs(predicted_mean - y_test)), np.mean(np.square(predicted_mean - y_test)), pool_cand_variance]


In [0]:
#Data Loading 
trainind_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/ALScripts_StartTrainSize150/"
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/"
X_train_all = np.load(data_path+"X_train.npy")
y_train_all = np.load(data_path+"y_train.npy")
X_test = np.load(data_path+"X_test.npy")
y_test = np.load(data_path+"y_test.npy")


In [0]:
#Active learning parameters/settings
num_experiments = 3
num_candidates = 5000
batch_size = 25
num_acquisitions = 2000

In [0]:
results_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/UTKFace/AL_Results_Start_150/"

#Looping across active learning experiments, each of which has a different initial training set
for e in range(1,3):
  train_data_indices = list(np.load(trainind_path+"trainindices"+str(e+1)+".npy"))
  #pool_indices = [i for i in range(y_train_all.shape[0]) if i not in train_data_indices]
  mae_file = "MaxVarMAEBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  mse_file = "MaxVarMSEBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  ind_file = "MaxVarIndBS"+str(batch_size)+"_Ind"+str(e+1)+".npy"
  exp_mse = []
  exp_mae = []
  #exp_mse = list(np.load(results_path+mse_file))
  #exp_mae = list(np.load(results_path+mae_file))
  #train_data_indices = list(np.load(results_path+ind_file))
  pool_indices = [i for i in range(y_train_all.shape[0]) if i not in train_data_indices]
  cand_indices = np.random.choice(pool_indices, num_candidates, replace=False)

  #Measuring initial MSE and MAE
  [mae, mse, cand_variances] = run_model(X_train_all[train_data_indices,:,:,:], y_train_all[train_data_indices], X_train_all[cand_indices,:,:,:])
  exp_mse.append(mse)
  exp_mae.append(mae)
  num_acquisitions = num_acquisitions - batch_size * (len(exp_mse)-1)

  #Iterating through active learning iterations
  for acq in range(num_acquisitions//batch_size):
    #Acquiring max. variance points
    new_train_ind = np.flip(np.argsort(cand_variances))[0:batch_size]
    for ind in new_train_ind:
      train_data_indices.append(cand_indices[ind])
      pool_indices.remove(cand_indices[ind])
    cand_indices = np.random.choice(pool_indices, num_candidates, replace=False)
    
    #Training model, evaluating model on test data, and saving performance
    [mae, mse, cand_variances] = run_model(X_train_all[train_data_indices,:,:,:], y_train_all[train_data_indices], X_train_all[cand_indices,:,:,:])
    exp_mae.append(mae)
    exp_mse.append(mse)
    print('Exp: ' + str(e+1) + ", # Training Pts: " + str(len(train_data_indices))+", Last 2 MSE: " + str(exp_mse[-2:]) + ", Len MSE: " + str(len(exp_mse)))

    np.save(results_path+mse_file,np.array(exp_mse))
    np.save(results_path+mae_file,np.array(exp_mae))
    np.save(results_path+ind_file,np.array(train_data_indices))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Exp: 2, # Training Pts: 175, Last 2 MSE: [320.9909442487526, 258.69422700026485], Len MSE: 2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Exp: 2, # Training Pts: 200, Last 2 MSE: [258.69422700026485, 242.96450289764334], Len MSE: 3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


...

Exp: 2, # Training Pts: 2125, Last 2 MSE: [160.26821741740028, 147.85901293811833], Len MSE: 80


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


Exp: 2, # Training Pts: 2150, Last 2 MSE: [147.85901293811833, 177.40278244839698], Len MSE: 81


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(150, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_regularizer=<keras.reg...)`


KeyboardInterrupt: ignored